# Chapter 6. Data Loading, Storage, and File Formats

In this chapter, we mostly work with data I/O, called data loading or parsing.

## Reading and Writing Data in Text Format

*pandas* has a number of built-in functions to read data in different formats:
- `read_csv`, `read_fwf`, `read_clipboard`, `read_excel`
- `read_html`, `read_xml`, `read_json`
- `read_parquet`, `read_pickle`, `read_sql`, `read_sql_table`

There are a number of common parameters:
- *Indexing* allows you to decide which columns to load, whether to get column names from the file, etc.
- *Type inference and data conversion* includes user-defined value conversions and custom list of missing value markers.
- *Data and time parsing* includes the ability to combine date and time information spread over multiple columns.
- *Interating* support for iterating over chunks of very large files.
- *Other data cleaning* techniques

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Loading a tiny sample data with header
df = pd.read_csv('examples/ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
# Loading a tiny sample data without header
pd.read_csv('examples/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [4]:
pd.read_csv('examples/ex2.csv', names=['a', 'b', 'c', 'd', 'message'], index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [5]:
pd.read_csv('examples/csv_mindex.csv', index_col=["key1", "key2"])

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [6]:
# Original code in the book receives some warning
# add \escape and engine to clarify
pd.read_csv("examples/ex3.txt", sep="\\s+", engine="python")

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


In [7]:
pd.read_csv("examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv("examples/ex5.csv")

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [9]:
# Setting up display
pd.options.display.max_rows = 10

# Load a large file by chunks
result = pd.read_csv("examples/ex6.csv")
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


In [10]:
# read five rows and examine first
pd.read_csv("examples/ex6.csv", nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


In [11]:
# read in chunks
chunker = pd.read_csv("examples/ex6.csv", chunksize=1000)
chunker # <pandas.io.parsers.readers.TextFileReader at 0x15e3cfa5950>

In [12]:
# frequency table for the "key" column
tot = pd.Series([], dtype='int64')

for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot.sum()

np.float64(10000.0)

In [13]:
# Writing data to text file
data = pd.read_csv("examples/ex5.csv")
data.to_csv("examples/out.csv")

In [14]:
import sys
data.to_csv(sys.stdout, sep="|", na_rep="NA")

|something|a|b|c|d|message
0|one|1|2|3.0|4|NA
1|two|5|6|NA|8|world
2|three|9|10|11.0|12|foo


In [15]:
data.to_csv(sys.stdout,index=False, columns=['a', 'b'])

a,b
1,2
5,6
9,10


In [16]:
# A JAON file
obj = """
{"name": "Wes",
 "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},
              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]
}
"""
obj

'\n{"name": "Wes",\n "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"],\n "pet": null,\n "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]},\n              {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]\n}\n'

In [17]:
# read json file
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'cities_lived': ['Akron', 'Nashville', 'New York', 'San Francisco'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 34, 'hobbies': ['guitars', 'soccer']},
  {'name': 'Katie', 'age': 42, 'hobbies': ['diving', 'art']}]}

In [18]:
# save to json
asjson = json.dumps(result)
asjson

'{"name": "Wes", "cities_lived": ["Akron", "Nashville", "New York", "San Francisco"], "pet": null, "siblings": [{"name": "Scott", "age": 34, "hobbies": ["guitars", "soccer"]}, {"name": "Katie", "age": 42, "hobbies": ["diving", "art"]}]}'

In [19]:
siblings = pd.DataFrame(result["siblings"], columns=["name", "age", "hobbies"])
siblings = siblings.set_index('name')
siblings

,age,hobbies
name,,
Scott,34,"[guitars, soccer]"
Katie,42,"[diving, art]"


In [20]:
# unpack the "hobbies" list column
hobbies_cols = pd.DataFrame(siblings["hobbies"].to_list(), index=siblings.index).add_prefix("hobby_")

# join to the original df
siblings = siblings.drop(columns="hobbies").join(hobbies_cols)
siblings

,age,hobby_0,hobby_1
name,,,
Scott,34,guitars,soccer
Katie,42,diving,art


In [21]:
ex_json = pd.read_json("examples/example.json")
ex_json

,a,b,c
0,1,2,3
1,4,5,6
2,7,8,9


In [22]:
ex_json.to_json(sys.stdout)

{"a":{"0":1,"1":4,"2":7},"b":{"0":2,"1":5,"2":8},"c":{"0":3,"1":6,"2":9}}

In [23]:
tables = pd.read_html("examples/fdic_failed_bank_list.html")
len(tables)
failures = tables[0]
failures.head()
close_timestamps = pd.to_datetime(failures["Closing Date"])
close_timestamps.dt.year.value_counts()

Closing Date
2010    157
2009    140
2011     92
2012     51
2008     25
       ... 
2004      4
2001      4
2007      3
2003      3
2000      2
Name: count, Length: 15, dtype: int64

In [24]:
from lxml import objectify
path = "pydata-book/datasets/mta_perf/Performance_MNR.xml"

In [25]:
# Read XML/HTML pages
path = r"../pydata-book/datasets/mta_perf/Performance_MNR.xml"
with open(path) as f:
    parsed = objectify.parse(f)

root = parsed.getroot()

data = []

skip_fields = ["PARENT_SEQ", "INDICATOR_SEQ",
               "DESIRED_CHANGE", "DECIMAL_PLACES"]

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

perf = pd.DataFrame(data)
perf.head()

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95.0,96.9,95.0,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95.0,96.0,95.0,95.0
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95.0,96.3,95.0,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95.0,96.8,95.0,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95.0,96.6,95.0,95.8


In [26]:
perf2 = pd.read_xml(path)
perf2.head(10)

,INDICATOR_SEQ,PARENT_SEQ,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,DESIRED_CHANGE,INDICATOR_UNIT,DECIMAL_PLACES,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,U,%,1,95.00,96.90,95.00,96.90
1,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,U,%,1,95.00,96.00,95.00,95.00
2,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,U,%,1,95.00,96.30,95.00,96.90
3,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,U,%,1,95.00,96.80,95.00,98.30
4,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,U,%,1,95.00,96.60,95.00,95.80
5,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,6,Service Indicators,M,U,%,1,95.00,96.20,95.00,94.40
6,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,7,Service Indicators,M,U,%,1,95.00,96.20,95.00,96.00
7,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,8,Service Indicators,M,U,%,1,95.00,96.20,95.00,96.40
8,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,9,Service Indicators,M,U,%,1,95.00,95.90,95.00,93.70
9,28445,NaN,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,10,Service Indicators,M,U,%,1,95.00,96.00,95.00,96.40


In [27]:
df.to_pickle("examples/df_pickle")
pd.read_pickle("examples/df_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [28]:
try:
    import pyarrow as pa
    print("pyarrow version:", pa.__version__)
except ImportError:
    print("pyarrow is NOT installed")


pyarrow version: 21.0.0


In [29]:
xlsx = pd.ExcelFile("examples/ex1.xlsx")
xlsx.sheet_names
xlsx.parse(sheet_name="Sheet1", index_col=0)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [30]:
writer = pd.ExcelWriter("examples/ex2.xlsx")
df.to_excel(writer, "Sheet1")

C:\Users\yluo\AppData\Local\Temp\ipykernel_1928\679774290.py:2: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, "Sheet1")


## Interacting with Web APIs

## Interacting with Databases

In [31]:
import sqlite3

In [32]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER);"""

In [33]:
con = sqlite3.connect('mydata.sqlite')
con.execute(query)

In [34]:
data = [
    ("Atlanta", "Georgia", 1.25, 6),
    ("Tallahassee", "Florida", 2.6, 3),
    ("Sacramento", "California", 1.7, 5),
    ("Boston", "Massachusett", -1.8, 12),
]
stmt = 'INSERT INTO test VALUES(?, ?, ?, ?)'

con.executemany(stmt, data)
con.commit()

In [35]:
cursor = con.execute('SELECT * FROM test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5),
 ('Boston', 'Massachusett', -1.8, 12)]

In [37]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [38]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
3,Boston,Massachusett,-1.80,12


In [40]:
import sqlalchemy as sqla